Fine-tuning a pretrained transformer BERT model for customized sentiment analysis using transformer PyTorch Trainer from Hugging Face

# Step 1: Install And Import Python Libraries

In step 1, we will install and import python libraries.

Firstly, let's install `transformers`, `datasets`, and `evaluate`.

In [1]:
# Install libraries
!pip install datasets transformers==4.28.0
!pip install transformers datasets evaluate

After installing the python packages, we will import the python libraries.
* `pandas` and `numpy` are imported for data processing.
* `tensorflow` and `transformers` are imported for modeling.
* `Dataset` is imported for the Hugging Face dataset format.
* `evaluate` is imported for model performance evaluation.

In [2]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate

# Step 2: Download And Read Data

The second step is to download and read the dataset.

The UCI Machine Learning Repository has the review data from three websites: imdb.com, amazon.com, and yelp.com. We will use the review data from amazon.com for this tutorial. Please follow these steps to download the data.
1. Go to: https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences
2. Click "Data Folder"
3. Download "sentiment labeled sentences.zip"
4. Unzip "sentiment labeled sentences.zip"
5. Copy the file "amazon_cells_labelled.txt" to your project folder

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory
import os
os.chdir("drive/My Drive/contents/nlp")

# Print out the current directory
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/contents/nlp


Now let's read the data into a `pandas` dataframe and see what the dataset looks like.

The dataset has two columns. One column contains the reviews and the other column contains the sentiment label for the review.

In [4]:
# Read in data
amz_review = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

# Take a look at the data
amz_review.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [5]:
# Get the dataset information
amz_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


The label value of 0 represents negative reviews and the label value of 1 represents positive reviews. The dataset has 500 positive reviews and 500 negative reviews. It is well-balanced, so we can use  accuracy as the metric to evaluate the model performance.

In [6]:
# Check the label distribution
amz_review['label'].value_counts()

0    500
1    500
Name: label, dtype: int64

# Step 3: Train Test Split

In step 3, we will split the dataset and have 80% as the training dataset and 20% as the testing dataset.

Using the `sample` method, we set `frac=0.8`, which randomly samples 80% of the data. `random_state=42` ensures that the sampling result is reproducible.

Dropping the `train_data` from the review dataset gives us the rest 20% of the data, which is our testing dataset.

In [7]:
# Training dataset
train_data = amz_review.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = amz_review.drop(train_data.index)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 800 records.
The testing dataset has 200 records.


After the train test split, there are 800 reviews in the training dataset and 200 reviews in the testing dataset.

# Step 4: Convert Pandas Dataframe to Hugging Face Dataset

In step 4, the training and the testing datasets will be converted from pandas dataframe to Hugging Face Dataset format.

Hugging Face Dataset objects are memory-mapped on drive, so they are not limited by RAM memory, which is very helpful for processing large datasets.

We use `Dataset.from_pandas` to convert a pandas dataframe to a Hugging Face Dataset.

In [8]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

The length of the Hugging Face Dataset is the same as the number of records in the pandas dataframe. For example, there are 800 records in the pandas dataframe for the training dataset, and the length of the converted Hugging Face Dataset for the training dataset is 800 too.

`hg_train_data[0]` gives us the first record in the Hugging Face Dataset. It is a dictionary with three keys, `review`, `label`, and `__index_level_0__`.
* `review` is the variable name for the review text. The name is inherited from the column name of the pandas dataframe.
* `label` is the variable name for the sentiment of the review text. The name is inherited from the column name of the pandas dataframe too.
* `__index_level_0__` is an automatically generated field from the pandas dataframe. It stores the index of the corresponding record.

In [9]:
# Length of the Dataset
print(f'The length of hg_train_data is {len(hg_train_data)}.\n')

# Check one review
hg_train_data[0]

The length of hg_train_data is 800.



{'review': 'Thanks again to Amazon for having the things I need for a good price!',
 'label': 1,
 '__index_level_0__': 521}

In this example, we can see that the review is `Thanks again to Amazon for having the things I need for a good price!`, the sentiment for the review is positive/1, and the index of this record is 521 in the pandas dataframe.

Checking the index 521 in the pandas dataframe confirms the same information with Hugging Face Dataset.

In [10]:
# Validate the record in pandas dataframe
amz_review.iloc[[521]]

,review,label
521,Thanks again to Amazon for having the things I...,1


# Step 5: Tokenize Text

In step 5, we will tokenize the review text using a tokenizer.

A tokenizer converts text into numbers to use as the input of the NLP (Natural Language Processing) models. Each number represents a token, which can be a word, part of a word, punctuation, or special tokens. How the text is tokenized is determined by the pretrained model. `AutoTokenizer.from_pretrained("bert-base-cased")` is used to download vocabulary from the pretrained `bert-base-cased` model, meaning that the text will be tokenized like a BERT model.

In [11]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

We can see that the tokenizer contains information such as model name, vocabulary size, max length, padding position, truncation position, and special tokens.

There are five special tokens for the BERT model. Other models may have different special tokens.
 * The tokens that are not part of the BERT model training dataset are unknown tokens. The unknown token is [UNK] and the ID for the unknown token is 100.
 * The separator token is [SEP] and the ID for the separator token is 102.
 * The pad token is [PAD] and the ID for the pad token is 0.
 * The sentence level classification token is [CLS] and the ID for the classification token is 101.
 * The mask token is [MASK] and the ID for the mask token is 103.

In [12]:
# Mapping between special tokens and their IDs.
print(f'The unknown token is {tokenizer.unk_token} and the ID for the unkown token is {tokenizer.unk_token_id}.')
print(f'The seperator token is {tokenizer.sep_token} and the ID for the seperator token is {tokenizer.sep_token_id}.')
print(f'The pad token is {tokenizer.pad_token} and the ID for the pad token is {tokenizer.pad_token_id}.')
print(f'The sentence level classification token is {tokenizer.cls_token} and the ID for the classification token is {tokenizer.cls_token_id}.')
print(f'The mask token is {tokenizer.mask_token} and the ID for the mask token is {tokenizer.mask_token_id}.')

The unknown token is [UNK] and the ID for the unkown token is 100.
The seperator token is [SEP] and the ID for the seperator token is 102.
The pad token is [PAD] and the ID for the pad token is 0.
The sentence level classification token is [CLS] and the ID for the classification token is 101.
The mask token is [MASK] and the ID for the mask token is 103.


After downloading the model vocabulary, the method `tokenizer` is used to tokenize the review corpus.
* `max_length` indicates the maximum number of tokens kept for each document.
 * If the document has more tokens than the `max_length`, it will be truncated.
 * If the document has less tokens than the `max_length`, it will be padded with zeros.
 * If `max_length` is unset or set to `None`, the maximum length from the pretrained model will be used. If the pretrained model does not have a maximum length parameter, `max_length` will be deactivated.
* `truncation` controls how the token truncation is implemented. `truncation=True` indicates that the truncation length is the length specified by `max_length`. If `max_length` is not specified, the max_length of the pretrained model is used.
* `padding` means adding zeros to shorter reviews in the dataset. The `padding` argument controls how `padding` is conducted.  
 * `padding=True` is the same as `padding='longest'`. It checks the longest sequence in the batch and pads zeros to that length. There is no padding if only one text document is provided.
 * `padding='max_length'` pads to `max_length` if it is specified, otherwise, it pads to the maximum acceptable input length for the model.
 * `padding=False` is the same as `padding='do_not_pad'`. It is the default, indicating that no padding is applied, so it can output a batch with sequences of different lengths.

In [13]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["review"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

After tokenization, we can see that both the training and the testing Dataset have 6 features, `'review'`, `'label'`, `'__index_level_0__'`, `'input_ids'`, `'token_type_ids'`, and `'attention_mask'`. The number of rows is stored with `num_rows`.

In [14]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['review', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 800
})
Dataset({
    features: ['review', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})


`dataset_train[0]` gives us the content for the first record in the training dataset in a dictionary format.
* `'review'` has the review text. The first review of the training dataset is `'Thanks again to Amazon for having the things I need for a good price!'`.
* `'label'` is the label of the classification. The first record is a positive review, so the label is 1.
* `'__index_level_0__'` is the index of the record. 521 means that the first record in the training dataset has the index 521 in the original pandas dataframe.
* `'input_ids'` are the IDs for the tokens. There are 32 token IDs because the `max_length` is 32 for the tokenization.
* `'token_type_ids'` is also called segment IDs.
 * BERT was trained on two tasks, Masked Language Modeling and Next Sentence Prediction. `'token_type_ids'` is for the Next Sentence Prediction, where two sentences are used to predict whether the second sentence is the next sentence for the first one.
 * The first sentence has all the tokens represented by zeros, and the second sentence has all the tokens represented by ones.
 * Because our classification task does not have a second sentence, all the values for `'token_type_ids'` are zeros.
* `'attention_mask'` indicates which token ID should get attention from the model, so the padding tokens are all zeros and other tokens are 1s.



In [15]:
# Check the first record
dataset_train[0]

{'review': 'Thanks again to Amazon for having the things I need for a good price!',
 'label': 1,
 '__index_level_0__': 521,
 'input_ids': [101,
  5749,
  1254,
  1106,
  9786,
  1111,
  1515,
  1103,
  1614,
  146,
  1444,
  1111,
  170,
  1363,
  3945,
  106,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

# Step 6: Load Pretrained Model

In step 6, we will load the pretrained model for sentiment analysis.

* `AutoModelForSequenceClassification` loads the BERT model without the sequence classification head.
* The method `from_pretrained()` loads the weights from the pretrained model into the new model, so the weights in the new model are not randomly initialized. Note that the new weights for the new sequence classification head are going to be randomly initialized.
* `bert-base-cased` is the name of the pretrained model. We can change it to a different model based on the nature of the project.
* `num_labels` indicates the number of classes. Our dataset has two classes, positive and negative, so `num_labels=2`.

In [16]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

# Step 7: Set Training Argument

In step 7, we will set the training arguments for the model.

Hugging Face has 96 parameters for `TrainingArguments`, which provides a lot of flexibility in fine-tuning the transfer learning model.
* `output_dir` is the directory to write the model checkpoints and model predictions.
* `logging_dir` is the directory for saving logs.
* `logging_strategy` is the strategy for logging the training information.
 * `'no'` means no logging for the training.
 * `'epoch'` means logging at the end of each epoch.
 * `'steps'` means logging at the end of each `logging_steps`.
* `logging_steps` is the number of steps between two logs. The default is 500.
* `num_train_epochs` is the total number of training epochs. The default value is 3.
* `per_device_train_batch_size` is the batch size per GPU/TPU core/CPU for training. The default value is 8.
* `per_device_eval_batch_size` is the batch size per GPU/TPU core/CPU for evaluation. The default value is 8.
* `learning_rate` is the initial learning rate for AdamW optimizer. The default value is 5e-5.
* `seed` is for reproducibility.
* `save_strategy` is the strategy for saving the checkpoint during training.
 * `'no'` means do not save during training.
 * `'epoch'` means saving at the end of each epoch.
 * `'steps'` means saving at the end of each `save_steps`. `'steps'` is the default value.
* `save_steps` is the number of steps before two checkpoint saves. The default value is 500.
* `evaluation_strategy` is the strategy for evaluation during training. It's helpful for us to monitor the model performance during model fine-tuning.
 * `'no'` means no evaluation during training.
 * `'epoch'` means evaluating at the end of each epoch and the evaluation results will be printed out at the end of each epoch.
 * `'steps'` means evaluating and reporting at the end of each `eval_steps`. `'no'` is the default value.
* `eval_steps` is the number of steps between two evaluations if `evaluation_strategy='steps'`. It defaults to the same value as `logging_steps` if not set.
* `load_best_model_at_end=True` indicates that the best model will be loaded at the end of the training. The default is `False`. When it is set to `True`, the `save_strategy` and `evaluation_strategy` must be the same. When both arguments are `'steps'`, the value of `save_steps` needs to be a round multiple of the value of `eval_steps`.

In [17]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",
    logging_dir='./sentiment_transfer_learning_transformer/logs',
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

# Step 8: Set Evaluation Metrics

In step 8, we will set the evaluation metric because Hugging Face Trainer does not evaluate the model performance automatically during the training process.

Hugging Face has an `evaluate` library with over 100 evaluation modules. We can see the list of all the modules using `evaluate.list_evaluation_modules()`.

In [18]:
# Number of evaluation modules
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 138 evaluation models in Hugging Face.



['precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'kaggle/ai4code',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'poseval',
 'brier_score',
 'abidlabs/mean_iou',
 'abidlabs/mean_iou2',
 'giulio98/codebleu',
 'mase',
 'mape',
 'smape',
 'dvitel/codebleu',
 'NCSOFT/harim_plus',
 'JP-SystemsX/nDCG',
 'Drunper/metrica_tesi',
 'jpxkqx/peak_signal_to_noise_ratio',
 'jpxkqx/signal_to_reconstruction_error',
 'hpi-dhc/FairEva

Since our dataset is highly balanced, we will use accuracy as the evaluation metric. It can be loaded using `evaluate.load("accuracy")`. After getting predictions from the model, the metric is computed using `metric.compute`.

In [20]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Step 9: Train Model Using Transformer Trainer

In step 9, we will train the model using the transformer `Trainer`.
* model is the model for training, evaluation, or prediction by the `Trainer`.
* `args` takes the arguments for tweaking the `Trainer`. It defaults to the instance of `TrainingArguments`.
* `train_dataset` is the training dataset name. If the dataset is in `Dataset` format, the unused columns will be automatically ignored. In our training dataset, `__index_level_0__` and `review` are not used by the model, so they are ignored.
* `eval_dataset` is the evaluation dataset name. Similar to the `train_dataset`, the unused columns will be automatically ignored for the `Dataset` format.
* `compute_metrics` takes the function for calculating evaluation metrics.
* `callbacks` takes a list of callbacks to customize the training loop. `EarlyStoppingCallback` stops the training by `early_stopping_patience` for the evaluation calls. There is no practical need to use early stopping because there are only two epochs for the model. It is included as an example code reference.

In [ ]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


We can see that the accuracy is above 90 percent in just 2 epochs.

# Step 10: Make Predictions for Text Classification

In step 10, we will talk about how to make predictions using the Hugging Face transformer Trainer model.

Passing the tokenized `Dataset` to the `.predict` method, we get the predictions for the customized transfer learning sentiment model. We can see that the prediction results contain multiple pieces of information.
* `Num examples = 200` indicates that there are 200 reviews in the testing dataset.
* `Batch size = 4` means that 4 reviews are processed each time.
* Under `PredictionOutput`, `predictions` has the logits for each class. logit is the last layer of the neural network before softmax is applied. `label_ids` has the actual labels. Please note that it is not predicted labels although it is under the `PredictionOutput`. We need to calculate the predicted labels based on the logit values.
* Under `metrics` there is information about the testing predictions.
 * `test_loss` is the loss for the testing dataset.  
 * `test_accuracy` is the percentage of correct predictions.
 * `test_runtime` is the runtime for testing.
 * `test_samples_per_second` is the number of samples the model can process in one second.
 * `test_steps_per_second` is the number of steps the model can process in one second.


In [ ]:
# Predictions
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
y_test_predict

The predicted logits for the transfer learning text classification model can be extracted using `.predictions`.

We can see that the prediction has two columns. The first column is the predicted logit for label 0 and the second column is the predicted logit for label 1. logit values do not sum up to 1.

In [ ]:
# Predicted logits
y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities
y_test_logits[:5]

To get the predicted probabilities, we need to apply softmax on the predicted logit values.

After applying softmax, we can see that the predicted probability for each review sums up to 1.

In [ ]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_logits)

# First 5 predicted logits
y_test_probabilities[:5]

To get the predicted labels, `argmax` is used to return the index of the maximum probability for each review, which corresponds to the labels of zeros and ones.

In [ ]:
# Predicted labels
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted probabilities
y_test_pred_labels[:5]

The actual labels can be extracted using `y_test_predict.label_ids`.

In [ ]:
# Actual labels
y_test_actual_labels = y_test_predict.label_ids

# First 5 predicted probabilities
y_test_actual_labels[:5]

# Step 11: Model Performance Evaluation

In step 11, we will make the transfer learning text classification model performance evaluation.

`trainer.evaluate` is a quick way to get the loss and the accuracy of the testing dataset.

We can see that the model has a loss of 0.28 and an accuracy of 91.5%.

In [ ]:
# Trainer evaluate
trainer.evaluate(dataset_test)

To calculate more model performance metrics, we can use `evaluate.load` to load the metrics of interest.

The results show that the testing dataset has a `f1` value of 0.91 and a `recall` value of 0.89.

In [ ]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

In [ ]:
# Load recall metric
metric_recall = evaluate.load("recall")

# Compute recall metric
metric_recall.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

# Step 12: Save and Load Model

In step 12, we will talk about how to save the model and reload it for prediction.

`tokenizer.save_pretrained` saves the tokenizer information to the drive and `model.save_model` saves the model to the drive.

In [ ]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_transformer/')

# Save model
trainer.save_model('./sentiment_transfer_learning_transformer/')

We can load the saved tokenizer later using `AutoTokenizer.from_pretrained()` and load the saved model using `AutoModelForSequenceClassification.from_pretrained()`.

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_transformer/")

# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_transformer/')